In [1]:
import subprocess
from datetime import datetime

# === Ejecutar el script
ruta_script = "../../scripts/core/run_backtest_heuristicoV2.py"
print(f"Ejecutando backtest: {ruta_script}")
inicio = datetime.now()

# Subprocess con salida en vivo
proceso = subprocess.Popen(["python", ruta_script], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# Mostrar salida en tiempo real
for linea in proceso.stdout:
    print(linea.strip())

# Confirmar fin
fin = datetime.now()
print(f"\nDuracion total: {fin - inicio}")


Ejecutando backtest: ../../scripts/core/run_backtest_heuristicoV2.py
Leyendo archivo de simbolos desde: C:\Users\leant\OneDrive\Documentos\trading\Jupyter\notebooks\test\config\json\symbols.json
Traceback (most recent call last):
File "C:\Users\leant\OneDrive\Documentos\trading\Jupyter\scripts\core\run_backtest_heuristicoV2.py", line 40, in <module>
SYMBOLS = json.load(f)["simbolos"]
File "C:\Users\leant\anaconda3\envs\Python_3912A\lib\json\__init__.py", line 293, in load
return loads(fp.read(),
File "C:\Users\leant\anaconda3\envs\Python_3912A\lib\json\__init__.py", line 346, in loads
return _default_decoder.decode(s)
File "C:\Users\leant\anaconda3\envs\Python_3912A\lib\json\decoder.py", line 337, in decode
obj, end = self.raw_decode(s, idx=_w(s, 0).end())
File "C:\Users\leant\anaconda3\envs\Python_3912A\lib\json\decoder.py", line 355, in raw_decode
raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)



In [2]:
import pandas as pd
from pathlib import Path
from datetime import date

# === Rutas ===
FECHA = date.today().isoformat()
BASE_PATH = Path("..").resolve().parent
BT_DIR = BASE_PATH / "reports" / "backtesting"
ARCHIVO = BT_DIR / f"bt_heuristicas_{FECHA}.csv"

# === Verificar existencia
if not ARCHIVO.exists():
    print(f"❌ Archivo no encontrado: {ARCHIVO}")
else:
    print(f"✅ Archivo encontrado: {ARCHIVO}")
    df = pd.read_csv(ARCHIVO)

    # === Validacion de columnas esperadas
    columnas_esperadas = ["simbolo", "estrategia", "fecha_entry", "fecha_exit", "signal", "entry", "exit", "retorno", "tp_hit", "sl_hit"]
    faltantes = [c for c in columnas_esperadas if c not in df.columns]

    if faltantes:
        print(f"❌ Faltan columnas requeridas: {faltantes}")
    else:
        print("✅ Todas las columnas requeridas estan presentes.")
        
        # === Tipos y resumen
        print("\n📊 Tipos de datos:")
        print(df.dtypes)

        print(f"\n📈 Total operaciones: {len(df)}")
        print(f"📆 Fechas: {df['fecha_entry'].min()} → {df['fecha_exit'].max()}")

        # === Señales por tipo
        print("\n🧭 Distribucion por tipo de señal:")
        print(df["signal"].value_counts())

        # === Duplicados
        dups = df.duplicated(subset=["simbolo", "estrategia", "fecha_entry", "signal"])
        print(f"\n🔁 Registros duplicados (estrategia + simbolo + fecha): {dups.sum()}")

        # === Valores nulos
        nulos = df.isna().sum()
        print("\n🕳️ Valores nulos por columna:")
        print(nulos[nulos > 0])

        # === Estrategias + simbolos unicos
        print(f"\n🧠 Estrategias unicas: {df['estrategia'].nunique()}")
        print(f"💡 Simbolos unicos: {df['simbolo'].nunique()}")
        print("\n🎯 Top combinaciones estrategia + simbolo:")
        display(df.groupby(["estrategia", "simbolo"]).size().sort_values(ascending=False).head(10))


✅ Archivo encontrado: C:\Users\leant\OneDrive\Documentos\trading\Jupyter\reports\backtesting\bt_heuristicas_2025-05-27.csv
✅ Todas las columnas requeridas estan presentes.

📊 Tipos de datos:
simbolo         object
estrategia      object
fecha_entry     object
fecha_exit      object
signal          object
entry          float64
exit           float64
retorno        float64
tp_hit            bool
sl_hit            bool
dtype: object

📈 Total operaciones: 69256
📆 Fechas: 2005-07-13 → 2025-05-23

🧭 Distribucion por tipo de señal:
sell    36551
buy     32705
Name: signal, dtype: int64

🔁 Registros duplicados (estrategia + simbolo + fecha): 0

🕳️ Valores nulos por columna:
Series([], dtype: int64)

🧠 Estrategias unicas: 13
💡 Simbolos unicos: 7

🎯 Top combinaciones estrategia + simbolo:


estrategia             simbolo
ma_envelope_reversals  AMD        3522
                       NVDA       3166
                       SMCI       2910
macd_hist_reversal     NVDA       2712
                       XOM        2711
                       AAPL       2658
                       MSFT       2657
                       AMD        2639
ma_envelope_reversals  AAPL       2615
                       TSLA       2564
dtype: int64

In [3]:
import pandas as pd
from pathlib import Path
from datetime import date

# === Ruta al archivo resumen ===
FECHA = date.today().isoformat()
BASE_PATH = Path("..").resolve().parent
BT_DIR = BASE_PATH / "reports" / "backtesting"
ARCHIVO = BT_DIR / f"resumen_metricas_{FECHA}.csv"

# === Verificar existencia
if not ARCHIVO.exists():
    print(f"❌ Archivo no encontrado: {ARCHIVO}")
else:
    print(f"✅ Archivo encontrado: {ARCHIVO}")
    df = pd.read_csv(ARCHIVO)

    # === Validar columnas requeridas
    columnas_requeridas = [
        "simbolo", "estrategia", "total_op", "winrate",
        "retorno_promedio", "retorno_total", "sharpe",
        "profit_factor", "max_drawdown"
    ]
    faltantes = [c for c in columnas_requeridas if c not in df.columns]

    if faltantes:
        print(f"❌ Faltan columnas en resumen: {faltantes}")
    else:
        print("✅ Todas las columnas requeridas estan presentes.")
        
        # === Tipos de datos
        print("\n📊 Tipos de datos:")
        print(df.dtypes)

        # === Tamaño y claves unicas
        print(f"\n🔢 Total filas: {len(df)}")
        print(f"🧠 Estrategias unicas: {df['estrategia'].nunique()}")
        print(f"💡 Simbolos unicos: {df['simbolo'].nunique()}")

        # === Verificar duplicados
        dups = df.duplicated(subset=["simbolo", "estrategia"])
        print(f"\n🔁 Registros duplicados estrategia+simbolo: {dups.sum()}")

        # === Verificar valores nulos
        nulos = df.isna().sum()
        print("\n🕳️ Valores nulos por columna:")
        print(nulos[nulos > 0])

        # === Top combinaciones
        print("\n🏆 Top estrategias por operaciones:")
        display(df.sort_values("total_op", ascending=False).head(10))

        print("\n📈 Top estrategias por Sharpe ratio:")
        display(df.sort_values("sharpe", ascending=False).head(10))

        print("\n📉 Top drawdown mas bajo:")
        display(df.sort_values("max_drawdown").head(10))


✅ Archivo encontrado: C:\Users\leant\OneDrive\Documentos\trading\Jupyter\reports\backtesting\resumen_metricas_2025-05-27.csv
✅ Todas las columnas requeridas estan presentes.

📊 Tipos de datos:
simbolo              object
estrategia           object
total_op              int64
winrate             float64
retorno_promedio    float64
retorno_total       float64
sharpe              float64
profit_factor       float64
max_drawdown        float64
dtype: object

🔢 Total filas: 91
🧠 Estrategias unicas: 13
💡 Simbolos unicos: 7

🔁 Registros duplicados estrategia+simbolo: 0

🕳️ Valores nulos por columna:
Series([], dtype: int64)

🏆 Top estrategias por operaciones:


,simbolo,estrategia,total_op,winrate,retorno_promedio,retorno_total,sharpe,profit_factor,max_drawdown
17,AMD,ma_envelope_reversals,3522,0.42,-0.000375,-1.3213,-0.01,0.98,-3.7937
43,NVDA,ma_envelope_reversals,3166,0.40,-0.003230,-10.2274,-0.06,0.86,-12.2081
56,SMCI,ma_envelope_reversals,2910,0.43,0.001661,4.8345,0.03,1.07,-3.5688
45,NVDA,macd_hist_reversal,2712,0.42,-0.000021,-0.0582,-0.00,1.00,-3.4139
84,XOM,macd_hist_reversal,2711,0.44,-0.000796,-2.1587,-0.02,0.94,-3.3416
6,AAPL,macd_hist_reversal,2658,0.46,0.001576,4.1883,0.04,1.10,-0.9073
32,MSFT,macd_hist_reversal,2657,0.42,-0.002253,-5.9855,-0.07,0.85,-6.5392
19,AMD,macd_hist_reversal,2639,0.44,0.002572,6.7877,0.04,1.11,-1.1692
4,AAPL,ma_envelope_reversals,2615,0.36,-0.003383,-8.8461,-0.09,0.82,-10.8603
69,TSLA,ma_envelope_reversals,2564,0.40,-0.003029,-7.7670,-0.05,0.89,-8.0683



📈 Top estrategias por Sharpe ratio:


,simbolo,estrategia,total_op,winrate,retorno_promedio,retorno_total,sharpe,profit_factor,max_drawdown
51,NVDA,volatilidad_contraria,32,0.62,0.026250,0.8400,0.45,2.88,0.1318
77,TSLA,volatilidad_contraria,44,0.59,0.036432,1.6030,0.44,2.75,0.1791
25,AMD,volatilidad_contraria,15,0.60,0.017767,0.2665,0.33,2.04,0.0944
65,TSLA,breakout_volumen,399,0.49,0.011503,4.5897,0.18,1.53,-0.9952
13,AMD,breakout_volumen,507,0.49,0.007965,4.0383,0.14,1.37,-0.0677
39,NVDA,breakout_volumen,493,0.47,0.007177,3.5384,0.13,1.37,-0.2276
11,AAPL,soporte_resistencia,1083,0.50,0.003408,3.6910,0.09,1.23,-0.4920
20,AMD,mov_avg,292,0.46,0.004368,1.2755,0.08,1.19,-0.2173
5,AAPL,macd_cruce,367,0.48,0.002986,1.0960,0.08,1.19,-0.1567
7,AAPL,mov_avg,266,0.48,0.002939,0.7817,0.08,1.19,-0.2532



📉 Top drawdown mas bajo:


,simbolo,estrategia,total_op,winrate,retorno_promedio,retorno_total,sharpe,profit_factor,max_drawdown
43,NVDA,ma_envelope_reversals,3166,0.40,-0.003230,-10.2274,-0.06,0.86,-12.2081
4,AAPL,ma_envelope_reversals,2615,0.36,-0.003383,-8.8461,-0.09,0.82,-10.8603
69,TSLA,ma_envelope_reversals,2564,0.40,-0.003029,-7.7670,-0.05,0.89,-8.0683
9,AAPL,rsi_reversion,1569,0.36,-0.004106,-6.4422,-0.11,0.77,-7.8395
74,TSLA,rsi_reversion,1090,0.38,-0.006366,-6.9387,-0.11,0.78,-7.0221
58,SMCI,macd_hist_reversal,2382,0.40,-0.002258,-5.3795,-0.04,0.91,-6.9950
32,MSFT,macd_hist_reversal,2657,0.42,-0.002253,-5.9855,-0.07,0.85,-6.5392
22,AMD,rsi_reversion,1484,0.40,-0.002126,-3.1550,-0.04,0.91,-4.6470
8,AAPL,rsi_divergencia,725,0.35,-0.005106,-3.7020,-0.14,0.72,-4.5298
48,NVDA,rsi_reversion,1322,0.42,-0.001833,-2.4231,-0.04,0.92,-3.7992
